## This notebook assumes that you have extracted the embeddings (using the procedure mentioned in the 0_ESM_Embeddings_Extractor.ipynb notebook) and have stored them in a zipped format

In [3]:
#### unzip train
!unzip -q <path_to_train_cdr3a.zip> -d  train_cdr3a
!unzip -q <path_to_train_cdr3b.zip> -d  train_cdr3b
!unzip -q <path_to_train_peptide.zip -d train_peptide

In [4]:
import sys
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from natsort import natsorted
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer,Input, Dense, Dropout, Activation, Concatenate, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
#from tensorflow.compat.v1 import InteractiveSession
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow.keras.metrics
####import tensorflow_addons as tfa not required
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
tf.random.set_seed(1)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical, plot_model
import torch
from tqdm import tqdm
import sklearn
import os
from natsort import natsorted
from sklearn.metrics import *

In [7]:

path_train_cdr3a = <'path_to_train_cdr3a_embeddings'>                  # Example ./train_cdr3a
path_train_cdr3b = <'path_to_train_cdr3b_embeddings'>                  # Example ./train_cdr3b
path_train_pepti = <'path_to_train_peptide_embeddings'>                # Example ./train_peptide

mat_cdr3a = os.listdir(path_train_cdr3a) 
mat_cdr3b = os.listdir(path_train_cdr3b) 
mat_pepti = os.listdir(path_train_pepti)

## natsort is used to order the pairs as they appear in the .csv; this would be helpful later to map the pairs with their respective labels

In [8]:
###train
mat_cdr3a = natsorted(mat_cdr3a)
mat_cdr3b = natsorted(mat_cdr3b)
mat_pepti = natsorted(mat_pepti)

## the following step extracts the embeddings and stores in a numpy matrix

In [13]:
# ESM
orig_matmat_cdr3a = np.zeros((samples,1280))
orig_matmat_cdr3b = np.zeros((samples,1280))
orig_matmat_pepti = np.zeros((samples,1280))


### load train samples 

for i in tqdm(range(samples)):

  x = torch.load(path_train_cdr3a+mat_cdr3a[i])['mean_representations'][33]
  orig_matmat_cdr3a[i] = x

  y = torch.load(path_train_cdr3b+mat_cdr3b[i])['mean_representations'][33]
  orig_matmat_cdr3b[i] = y

  z = torch.load(path_train_pepti+mat_pepti[i])['mean_representations'][33]
  orig_matmat_pepti[i] = z

### load train samples 

# for i in tqdm(range(samples)):

#   orig_matmat_cdr3a[i] = np.load(path_train_cdr3a+mat_cdr3a[i])
#   orig_matmat_cdr3b[i] = np.load(path_train_cdr3b+mat_cdr3b[i])
#   orig_matmat_pepti[i] = np.load(path_train_pepti+mat_pepti[i])



100%|██████████| 16464/16464 [00:15<00:00, 1063.16it/s]


## load *.csv files for labels

In [14]:
df_train = pd.read_csv(<path_to_csv_file_with_labels>)
y_train = df_train['binder'].values
orig_y_train = y_train.reshape(-1,1)

In [18]:
### model
def clear_sess():
  try:
    del model 
    del history 
  except:
    pass
  from tensorflow.keras import backend as K
  K.clear_session()
  import gc
  gc.collect()



  return None


def keras_mcc(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

## model MLP for CDR3b and peptide || CDR3a and peptide

In [ ]:
clear_sess()
#input_1


input_1 = Input(shape = (1280,), name='i_1')
dense1_1 = Dense(128, activation = 'elu')(input_1)
bn1_1 = BatchNormalization()(dense1_1)
drop_1 = Dropout(0.5)(bn1_1)

#input_2
input_2 = Input(shape = (1280,), name='i_2')
dense2_1 = Dense(128, activation = 'elu')(input_2)
bn2_1 = BatchNormalization()(dense2_1)
drop_2 = Dropout(0.5)(bn2_1)

# #input_3
# input_3 = Input(shape = (1024,), name='i_3')
# dense3_1 = Dense(512, activation = 'relu')(input_3)
# bn3_1 = BatchNormalization()(dense3_1)
# drop_3 = Dropout(0.5)(bn3_1)


# concatenate
##concat   = Concatenate()([dense1_1, dense2_1])
##concat   = Concatenate()([bn1_1, bn2_1])
concat   = Concatenate()([drop_1,drop_2])
fc_1   = Dense(512, activation = 'relu')(concat)
drop_4 = Dropout(0.5)(fc_1)
fc_2   = Dense(256, activation = 'relu')(drop_4)
#classification output- TCR-Peptide Binding yes/no
output  = Dense(1, activation = 'sigmoid')(fc_2)
 
# create model with two inputs
model = Model(inputs=[input_1,input_2], outputs=output)

## model MLP ## for CDR3a with CDR3b and peptide

In [ ]:
clear_sess()
#input_1

input_1 = Input(shape = (1280,), name='i_1')
dense1_1 = Dense(128, activation = 'relu')(input_1)
bn1_1 = BatchNormalization()(dense1_1)

#input_2
input_2 = Input(shape = (1280,), name='i_2')
dense2_1 = Dense(128, activation = 'relu')(input_2)
bn2_1 = BatchNormalization()(dense2_1)

#input_3
input_3 = Input(shape = (1280,), name='i_3')
dense3_1 = Dense(128, activation = 'relu')(input_3)
bn3_1 = BatchNormalization()(dense3_1)
 
# concatenate
##concat   = Concatenate()([dense1_1, dense2_1])
concat   = Concatenate()([bn1_1, bn2_1, bn3_1])
fc_1   = Dense(128, activation = 'relu')(concat)
#drop_1 = Dropout(0.5)(fc_1)
fc_2   = Dense(128, activation = 'relu')(fc_1)
#classification output
output  = Dense(1, activation = 'sigmoid')(fc_2)
 
# create model with two inputs
model = Model(inputs=[input_1,input_2, input_3], outputs=output)

In [ ]:
metrics_c = [tensorflow.keras.metrics.AUC(name="auc_roc",curve="ROC"),tensorflow.keras.metrics.AUC(name="auc_pr",curve="PR"),keras_mcc]

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.008), 
              metrics=metrics_c)

In [ ]:
early_stop = EarlyStopping(monitor='loss',min_delta=0,patience=10, verbose=0,mode='min',restore_best_weights=True)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_keras_mcc', factor=0.99,patience=20, min_lr=0.005, verbose=1)

In [ ]:
checkpoint_filepath_1 = 'weights-improvement-val-auc-pr.hdf5'
model_checkpoint_callback_1 = ModelCheckpoint(filepath=checkpoint_filepath_1,save_weights_only=False,monitor='val_auc_pr',mode='max',save_best_only=True)

checkpoint_filepath_2 = 'weights-improvement-val-keras-mcc.hdf5'
model_checkpoint_callback_2 = ModelCheckpoint(filepath=checkpoint_filepath_2,save_weights_only=False,monitor='val_keras_mcc',mode='max',save_best_only=True)

## uncomment the following cell for running CDR3a, CDR3b, peptide model

In [ ]:
# fit the keras model on the dataset CDR3a, CDR3b, peptide
# history=model.fit([train_cdr3a, train_cdr3b,train_pep],train_Y,
#                   batch_size=1024, epochs=500,
#                   validation_split=0.1,
#                   callbacks=[model_checkpoint_callback_1, model_checkpoint_callback_2, reduce_lr ]
#                   verbose=1)

## uncomment the following cell for running CDR3b, peptide model

In [ ]:
# # fit the keras model on the dataset CDR3b, peptide
# history=model.fit([matmat_cdr3b,matmat_pepti],y_train,
#                   batch_size=1024, epochs=500,
#                   validation_split=0.1,
#                   callbacks=[model_checkpoint_callback_1, model_checkpoint_callback_2, reduce_lr ]
#                   verbose=0)

## uncomment the following cell for running CDR3a, peptide model

In [ ]:
# # fit the keras model on the dataset CDR3a, CDR3b, peptide
# history=model.fit([matmat_cdr3a,matmat_pepti],y_train,
#                   batch_size=1024, epochs=500,
#                   verbose=0,
#                   validation_split=0.1,
#                   callbacks=[model_checkpoint_callback_1, model_checkpoint_callback_2, reduce_lr ]
#                   )

## once the best model is trained, we can test it over the evaluation dataset

In [ ]:
# ####model load max aucpr
model_loaded = '/content/weights-improvement-val-auc-pr.hdf5'
model = tensorflow.keras.models.load_model(model_loaded,compile=False)
y_pred = model.predict([evalmatmat_cdr3a, evalmatmat_cdr3b,evalmatmat_pepti])
#y_pred = model.predict([evalmatmat_cdr3b,evalmatmat_pepti])
y_act = y_eval.flatten()
y_pred= y_pred.flatten()
y_pred_c=np.where(y_pred>0.5,1,0)
print(roc_auc_score(y_act, y_pred),average_precision_score(y_act, y_pred),matthews_corrcoef(y_act,y_pred_c),cohen_kappa_score(y_act,y_pred_c))

In [ ]:
# ####model load max mcc
model_loaded = '/content/weights-improvement-val-keras-mcc.hdf5'
model = tensorflow.keras.models.load_model(model_loaded,compile=False)
y_pred = model.predict([evalmatmat_cdr3a, evalmatmat_cdr3b,evalmatmat_pepti])
#y_pred = model.predict([evalmatmat_cdr3b,evalmatmat_pepti])
y_act = y_eval.flatten()
y_pred= y_pred.flatten()
y_pred_c=np.where(y_pred>0.5,1,0)
print(roc_auc_score(y_act, y_pred),average_precision_score(y_act, y_pred),matthews_corrcoef(y_act,y_pred_c),cohen_kappa_score(y_act,y_pred_c))

## once the model training process is okay, we can run it for the CV set as originally defined by the netTCR 2.0 work

In [ ]:
fold_assignments = df_train['partition']

# Your data and labels (replace with your own data)
X = orig_matmat_pepti  # Your data
y = orig_y_train  # Your labels

# Number of predefined folds (assumes the folds are numbered 1, 2, 3, 4, 5)
num_folds = 5

for fold in range(1, num_folds + 1):
    # Split data into train and test sets based on the predefined test fold
    X_test = X[np.array(fold_assignments) == fold]
    y_test = y[np.array(fold_assignments) == fold]
    data_test = df_train[np.array(fold_assignments) == fold]

    X_train = X[np.array(fold_assignments) != fold]
    y_train = y[np.array(fold_assignments) != fold]
    data_train = df_train[np.array(fold_assignments) != fold]

    # Replace this with your machine learning model and evaluation code
    # Train your model on X_train, y_train, and evaluate it on X_test, y_test

    print(f"Fold {fold}: Train {len(X_train)}, Test {len(X_test)}")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings

# Suppress all UserWarnings
warnings.simplefilter(action='ignore', category=UserWarning)

## AB chain

In [ ]:
### this is for CV originally defined

def make_data_alphabeta(orig_matmat_cdr3a, orig_matmat_cdr3b, orig_matmat_pepti, orig_y_train, fold_assignments, fold):

    acti = 'gelu'
    # print(fold)
    matmat_cdr3a = orig_matmat_cdr3a[np.array(fold_assignments) != fold]
    matmat_cdr3b = orig_matmat_cdr3b[np.array(fold_assignments) != fold]
    matmat_pepti = orig_matmat_pepti[np.array(fold_assignments) != fold]
    y_train      = orig_y_train[np.array(fold_assignments) != fold]
    data_train = df_train[np.array(fold_assignments) != fold]

    #### internal eval
    evalmatmat_cdr3a = orig_matmat_cdr3a[np.array(fold_assignments) == fold]
    evalmatmat_cdr3b = orig_matmat_cdr3b[np.array(fold_assignments) == fold]
    evalmatmat_pepti = orig_matmat_pepti[np.array(fold_assignments) == fold]
    y_eval           = orig_y_train[np.array(fold_assignments) == fold]
    data_test = df_train[np.array(fold_assignments) == fold]
    
    train_input = [matmat_cdr3a,matmat_cdr3b,matmat_pepti]
    train_output= y_train


    test_input = [evalmatmat_cdr3a,evalmatmat_cdr3b,evalmatmat_pepti]
    test_output= y_eval


    #input 1
    input_1 = Input(shape = (1280,), name='i_1')
    dense1_1 = Dense(128, activation = acti)(input_1)
    bn1_1 = BatchNormalization()(dense1_1)
    drop_1 = Dropout(0.5)(bn1_1)

    #input_2
    input_2 = Input(shape = (1280,), name='i_2')
    dense2_1 = Dense(128, activation = acti)(input_2)
    bn2_1 = BatchNormalization()(dense2_1)
    drop_2 = Dropout(0.5)(bn2_1)

    #input_3
    input_3 = Input(shape = (1280,), name='i_3')
    dense3_1 = Dense(128, activation = acti)(input_3)
    bn3_1 = BatchNormalization()(dense3_1)
    drop_3 = Dropout(0.5)(bn3_1)

    # concatenate
    ##concat   = Concatenate()([dense1_1, dense2_1])
    ##concat   = Concatenate()([bn1_1, bn2_1])
    concat   = Concatenate()([drop_1,drop_2,drop_3])
    fc_1   = Dense(256, activation = acti)(concat)
    drop_4 = Dropout(0.5)(fc_1)
    fc_2   = Dense(128, activation = acti)(drop_4)
    #classification output- TCR-Peptide Binding yes/no
    output  = Dense(1, activation = 'sigmoid')(fc_2)

    # create model with two inputs
    model = Model(inputs=[input_1,input_2, input_3], outputs=output)

    return model, train_input, test_input, train_output, test_output, data_train, data_test

## B chain

In [ ]:
### this is for CV originally defined

def make_data_beta(orig_matmat_cdr3b, orig_matmat_pepti, orig_y_train, fold_assignments, fold):

    acti = 'gelu'
    matmat_cdr3b = orig_matmat_cdr3b[np.array(fold_assignments) != fold]
    matmat_pepti = orig_matmat_pepti[np.array(fold_assignments) != fold]
    y_train      = orig_y_train[np.array(fold_assignments) != fold]
    data_train = df_train[np.array(fold_assignments) != fold]

    #### internal eval
    evalmatmat_cdr3b = orig_matmat_cdr3b[np.array(fold_assignments) == fold]
    evalmatmat_pepti = orig_matmat_pepti[np.array(fold_assignments) == fold]
    y_eval           = orig_y_train[np.array(fold_assignments) == fold]
    data_test = df_train[np.array(fold_assignments) == fold]

    train_input = [matmat_cdr3b,matmat_pepti]
    train_output= y_train

    test_input = [evalmatmat_cdr3b,evalmatmat_pepti]
    test_output= y_eval
    
    #input 1
    input_1 = Input(shape = (1280,), name='i_1')
    dense1_1 = Dense(128, activation = acti)(input_1)
    bn1_1 = BatchNormalization()(dense1_1)
    drop_1 = Dropout(0.5)(bn1_1)

    #input_2
    input_2 = Input(shape = (1280,), name='i_2')
    dense2_1 = Dense(128, activation = acti)(input_2)
    bn2_1 = BatchNormalization()(dense2_1)
    drop_2 = Dropout(0.5)(bn2_1)

    # concatenate
    concat   = Concatenate()([drop_1,drop_2])
    fc_1   = Dense(256, activation = acti)(concat)
    drop_4 = Dropout(0.5)(fc_1)
    fc_2   = Dense(128, activation = acti)(drop_4)
    #classification output- TCR-Peptide Binding yes/no
    output  = Dense(1, activation = 'sigmoid')(fc_2)

    # create model with two inputs
    model = Model(inputs=[input_1,input_2], outputs=output)

    return model, train_input, test_input, train_output, test_output, data_train, data_test

## A chain

In [ ]:
### this is for CV originally defined

def make_data_alpha(orig_matmat_cdr3a, orig_matmat_pepti, orig_y_train, fold_assignments, fold):

    acti = 'gelu'
    # print(fold)
    matmat_cdr3a = orig_matmat_cdr3a[np.array(fold_assignments) != fold]
    matmat_pepti = orig_matmat_pepti[np.array(fold_assignments) != fold]
    y_train      = orig_y_train[np.array(fold_assignments) != fold]
    data_train = df_train[np.array(fold_assignments) != fold]

    #### internal eval
    evalmatmat_cdr3a = orig_matmat_cdr3a[np.array(fold_assignments) == fold]
    evalmatmat_pepti = orig_matmat_pepti[np.array(fold_assignments) == fold]
    y_eval           = orig_y_train[np.array(fold_assignments) == fold]
    data_test = df_train[np.array(fold_assignments) == fold]

    train_input = [matmat_cdr3a,matmat_pepti]
    train_output= y_train

    test_input = [evalmatmat_cdr3a,evalmatmat_pepti]
    test_output= y_eval
    
    #input 1
    input_1 = Input(shape = (1280,), name='i_1')
    dense1_1 = Dense(128, activation = acti)(input_1)
    bn1_1 = BatchNormalization()(dense1_1)
    drop_1 = Dropout(0.5)(bn1_1)

    #input_2
    input_2 = Input(shape = (1280,), name='i_2')
    dense2_1 = Dense(128, activation = acti)(input_2)
    bn2_1 = BatchNormalization()(dense2_1)
    drop_2 = Dropout(0.5)(bn2_1)

    # concatenate
    concat   = Concatenate()([drop_1,drop_2])
    fc_1   = Dense(256, activation = acti)(concat)
    drop_4 = Dropout(0.5)(fc_1)
    fc_2   = Dense(128, activation = acti)(drop_4)
    #classification output- TCR-Peptide Binding yes/no
    output  = Dense(1, activation = 'sigmoid')(fc_2)

    # create model with two inputs
    model = Model(inputs=[input_1,input_2], outputs=output)

    return model, train_input, test_input, train_output, test_output, data_train, data_test

# Model Runs - Run AB chain

In [ ]:
fold_assignments = df_train['partition']

for fold in range(1, num_folds + 1):

    clear_sess()
    model, train_input, test_input, train_output, test_output, data_train, data_test = make_data_alphabeta(orig_matmat_cdr3a, orig_matmat_cdr3b, orig_matmat_pepti, orig_y_train, fold_assignments, fold)

    print(f"Fold {fold}: Train {train_input[0].shape}, Test {test_input[0].shape}")

    metrics_c = [tensorflow.keras.metrics.AUC(name="auc_pr",curve="PR"),tensorflow.keras.metrics.AUC(name="auc_roc",curve="ROC"),keras_mcc]
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.007), metrics=metrics_c )
    reduce_lr = ReduceLROnPlateau(monitor='val_keras_mcc', factor=0.99,patience=20, min_lr=0.005, verbose=0)
    checkpoint_filepath_mcc = f'/content/weights/weights-improvement-{fold}-mcc.hdf5'
    checkpoint_filepath_pr = f'/content/weights/weights-improvement-{fold}-pr.hdf5'
    model_checkpoint_callback_mcc = ModelCheckpoint(filepath=checkpoint_filepath_mcc,save_weights_only=False,monitor='val_keras_mcc',mode='max',save_best_only=True)
    model_checkpoint_callback_pr = ModelCheckpoint(filepath=checkpoint_filepath_pr,save_weights_only=False,monitor='val_auc_pr',mode='max',save_best_only=True)


    history = model.fit(train_input, train_output,epochs=500,batch_size=32, verbose=0, validation_data=(test_input,test_output),
                        callbacks=[model_checkpoint_callback_mcc,model_checkpoint_callback_pr,reduce_lr])

    peplist = ['GILGFVFTL', 'GLCTLVAML', 'NLVPMVATV', 'LLFGYPVYV', 'FLYALALLL', 'RTLNAWVKV', 'IMDQVPFSV', 'KTWGQYWQV']
    print('Loading best MCC model')
    model_loaded = checkpoint_filepath_mcc
    model = tensorflow.keras.models.load_model(model_loaded,compile=False)

    y_pred = model.predict(test_input, verbose=0)
    y_act = test_output.flatten()
    y_pred= y_pred.flatten()
    y_pred_c=np.where(y_pred>0.5,1,0)
    print('MCC: ', matthews_corrcoef(y_act,y_pred_c), 'ROC :', roc_auc_score(y_act, y_pred), 'PR: ', average_precision_score(y_act, y_pred),  'F1:', f1_score(y_act,y_pred_c))
    for pep in peplist:
            y_act_pep   = y_act[data_test['peptide']  == pep]
            y_pred_pep  = y_pred[data_test['peptide'] == pep]
            y_pred_c_pep=y_pred_c[data_test['peptide']== pep]
            print(f'{pep}:', 'MCC: ', matthews_corrcoef(y_act_pep,y_pred_c_pep), 'ROC :', roc_auc_score(y_act_pep, y_pred_pep), 'PR: ', average_precision_score(y_act_pep, y_pred_pep), 'F1:', f1_score(y_act_pep,y_pred_c_pep))

    print('Loading best PR model')
    model_loaded = checkpoint_filepath_pr
    model = tensorflow.keras.models.load_model(model_loaded,compile=False)

    y_pred = model.predict(test_input, verbose=0)
    y_act = test_output.flatten()
    y_pred= y_pred.flatten()
    y_pred_c=np.where(y_pred>0.5,1,0)
    print('MCC: ', matthews_corrcoef(y_act,y_pred_c), 'ROC :', roc_auc_score(y_act, y_pred), 'PR: ', average_precision_score(y_act, y_pred),  'F1:', f1_score(y_act,y_pred_c))

    for pep in peplist:
        y_act_pep   = y_act[data_test['peptide']  == pep]
        y_pred_pep  = y_pred[data_test['peptide'] == pep]
        y_pred_c_pep=y_pred_c[data_test['peptide']== pep]
        print(f'{pep}:', 'MCC: ', matthews_corrcoef(y_act_pep,y_pred_c_pep), 'ROC :', roc_auc_score(y_act_pep, y_pred_pep), 'PR: ', average_precision_score(y_act_pep, y_pred_pep), 'F1:', f1_score(y_act_pep,y_pred_c_pep))

# Model runs - B chain

In [ ]:
fold_assignments = df_train['partition']

# Number of predefined folds (assumes the folds are numbered 1, 2, 3, 4, 5)
num_folds = 5

for fold in range(1, num_folds + 1):
    clear_sess()
    model, train_input, test_input, train_output, test_output, data_train, data_test = make_data_beta(orig_matmat_cdr3b, orig_matmat_pepti, orig_y_train, fold_assignments, fold)

    print(f"Fold {fold}: Train {train_input[0].shape}, Test {test_input[0].shape}")

    metrics_c = [tensorflow.keras.metrics.AUC(name="auc_pr",curve="PR"),tensorflow.keras.metrics.AUC(name="auc_roc",curve="ROC"),keras_mcc]
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.007), metrics=metrics_c )
    reduce_lr = ReduceLROnPlateau(monitor='val_keras_mcc', factor=0.99,patience=20, min_lr=0.005, verbose=0)
    checkpoint_filepath_mcc = f'/content/weights/weights-improvement-{fold}-mcc.hdf5'
    checkpoint_filepath_pr = f'/content/weights/weights-improvement-{fold}-pr.hdf5'
    model_checkpoint_callback_mcc = ModelCheckpoint(filepath=checkpoint_filepath_mcc,save_weights_only=False,monitor='val_keras_mcc',mode='max',save_best_only=True)
    model_checkpoint_callback_pr = ModelCheckpoint(filepath=checkpoint_filepath_pr,save_weights_only=False,monitor='val_auc_pr',mode='max',save_best_only=True)

    history = model.fit(train_input, train_output,epochs=200,batch_size=64, verbose=0, validation_data=(test_input,test_output),
                      callbacks=[model_checkpoint_callback_mcc,model_checkpoint_callback_roc,model_checkpoint_callback_pr,reduce_lr])
    print('Loading best MCC model')
    model_loaded = checkpoint_filepath_mcc
    model = tensorflow.keras.models.load_model(model_loaded,compile=False)

    y_pred = model.predict(test_input, verbose=0)
    y_act = test_output.flatten()
    y_pred= y_pred.flatten()
    y_pred_c=np.where(y_pred>0.5,1,0)
    print('MCC: ', matthews_corrcoef(y_act,y_pred_c), 'ROC :', roc_auc_score(y_act, y_pred), 'PR: ', average_precision_score(y_act, y_pred), 'F1:', f1_score(y_act,y_pred_c))

    print('Loading best PR model')
    model_loaded = checkpoint_filepath_pr
    model = tensorflow.keras.models.load_model(model_loaded,compile=False)

    y_pred = model.predict(test_input, verbose=0)
    y_act = test_output.flatten()
    y_pred= y_pred.flatten()
    y_pred_c=np.where(y_pred>0.5,1,0)
    print('MCC: ', matthews_corrcoef(y_act,y_pred_c), 'ROC :', roc_auc_score(y_act, y_pred), 'PR: ', average_precision_score(y_act, y_pred), 'F1:', f1_score(y_act,y_pred_c))

# Model runs A chain

In [ ]:
fold_assignments = df_train['partition']

# Number of predefined folds (assumes the folds are numbered 1, 2, 3, 4, 5)
num_folds = 5

for fold in range(1, num_folds + 1):
    clear_sess()
    model, train_input, test_input, train_output, test_output, data_train, data_test = make_data_alpha(orig_matmat_cdr3a, orig_matmat_pepti, orig_y_train, fold_assignments, fold)

    print(f"Fold {fold}: Train {train_input[0].shape}, Test {test_input[0].shape}")

    metrics_c = [tensorflow.keras.metrics.AUC(name="auc_pr",curve="PR"),tensorflow.keras.metrics.AUC(name="auc_roc",curve="ROC"),keras_mcc]
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.007), metrics=metrics_c )
    reduce_lr = ReduceLROnPlateau(monitor='val_keras_mcc', factor=0.99,patience=20, min_lr=0.005, verbose=0)
    checkpoint_filepath_mcc = f'/content/weights/weights-improvement-{fold}-mcc.hdf5'
    checkpoint_filepath_pr = f'/content/weights/weights-improvement-{fold}-pr.hdf5'
    model_checkpoint_callback_mcc = ModelCheckpoint(filepath=checkpoint_filepath_mcc,save_weights_only=False,monitor='val_keras_mcc',mode='max',save_best_only=True)
    model_checkpoint_callback_pr = ModelCheckpoint(filepath=checkpoint_filepath_pr,save_weights_only=False,monitor='val_auc_pr',mode='max',save_best_only=True)

    history = model.fit(train_input, train_output,epochs=200,batch_size=2048, verbose=0, validation_data=(test_input,test_output),
                      callbacks=[model_checkpoint_callback_mcc,model_checkpoint_callback_roc,model_checkpoint_callback_pr,reduce_lr])
    print('Loading best MCC model')
    model_loaded = checkpoint_filepath_mcc
    model = tensorflow.keras.models.load_model(model_loaded,compile=False)

    y_pred = model.predict(test_input, verbose=0)
    y_act = test_output.flatten()
    y_pred= y_pred.flatten()
    y_pred_c=np.where(y_pred>0.5,1,0)
    print('MCC: ', matthews_corrcoef(y_act,y_pred_c), 'ROC :', roc_auc_score(y_act, y_pred), 'PR: ', average_precision_score(y_act, y_pred), 'F1:', f1_score(y_act,y_pred_c))

    print('Loading best PR model')
    model_loaded = checkpoint_filepath_pr
    model = tensorflow.keras.models.load_model(model_loaded,compile=False)

    y_pred = model.predict(test_input, verbose=0)
    y_act = test_output.flatten()
    y_pred= y_pred.flatten()
    y_pred_c=np.where(y_pred>0.5,1,0)
    print('MCC: ', matthews_corrcoef(y_act,y_pred_c), 'ROC :', roc_auc_score(y_act, y_pred), 'PR: ', average_precision_score(y_act, y_pred), 'F1:', f1_score(y_act,y_pred_c))